In [76]:
import requests
import json
import numpy as np
import pandas as pd
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")
today = datetime.now()

def odds_conv(odds):
    """
    Convert odds to implied probabilities.

    Args:
        odds (float): The odds to convert.

    Returns:
        float: The implied probability.
    """
    if odds > 0:
        return 1 / (odds / 100 + 1)
    else:
        return 1 / (1 + 100 / abs(odds))

def fair_odds(odds_a, odds_b):
    """
    Calculate fair odds for two events.

    Args:
        odds_a (float): The odds for event A.
        odds_b (float): The odds for event B.

    Returns:
        tuple: A tuple containing the fair odds for event A and event B.
    """
    fair_a = odds_conv(odds_a) / (odds_conv(odds_a) + odds_conv(odds_b))
    fair_b = odds_conv(odds_b) / (odds_conv(odds_a) + odds_conv(odds_b))
    return (fair_a, fair_b)

def avg(x, y):
    """
    Calculate the average of two numbers.

    Args:
        x (float): The first number.
        y (float): The second number.

    Returns:
        float: The average of x and y.
    """
    return (x + y) / 2

def profit(stake, odds):
    """
    Calculate profit from a bet.

    Args:
        stake (float): The amount of the stake.
        odds (float): The odds for the bet.

    Returns:
        float: The profit from the bet.
    """
    if odds > 0:
        return odds / 100 * stake
    else:
        return 100 / abs(odds) * stake

def kelly_criterion(b, p, q):
    """
    Calculate the Kelly Criterion for optimal betting.

    Args:
        b (float): The bankroll.
        p (float): The probability of winning.
        q (float): The probability of losing.

    Returns:
        float: The optimal bet size according to the Kelly Criterion.
    """
    return (b * p - q) / b

def width(a, b):
    """
    Calculate the width between two numbers.

    Args:
        a (float): The first number.
        b (float): The second number.

    Returns:
        int: The absolute width between a and b.
    """
    if a < 0 and b < 0:
        return abs(int(str(a)[-2:]) + int(str(b)[-2:]))
    else:
        return abs(int(str(a)[-2:]) - int(str(b)[-2:]))

def get_sport_odds(api_key, sport_key='upcoming', regions='us', oddsFormat='american'):
    """
    Fetches sports odds data from the Odds API.

    Args:
        api_key (str): API key for authentication.
        sport_key (str): Key identifying the specific sport.
        regions (str): Region for which odds are to be fetched.

    Returns:
        dict: JSON data containing odds information for the specified sport.
    """
    base_url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds'
    params = {
        'apiKey': api_key,
        'regions': regions,
        'oddsFormat': oddsFormat,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f'Failed to retrieve data. Status code: {response.status_code}')
        return None

def process_odds_data(data):
    """
    Processes the odds data and creates a DataFrame.

    Args:
        data (dict): JSON data containing odds information.

    Returns:
        pd.DataFrame: Processed data in the form of a DataFrame.
    """
    data_list = []
    for game in data:
        game_date = game['commence_time']
        sport = game['sport_title']
        home_team = game['home_team']
        away_team = game['away_team']
        for bookmaker in game['bookmakers']:
            bookmaker_name = bookmaker['title']
            for market in bookmaker['markets']:
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        team_name = outcome['name']
                        if team_name.lower() == home_team.lower():
                            team_type = 'home'
                        elif team_name.lower() == away_team.lower():
                            team_type = 'away'
                        else:
                            team_type = 'unknown'
                        row = {
                            'date': game_date,
                            'sport': sport,
                            'home_team': home_team,
                            'away_team': away_team,
                            'bookmaker': bookmaker_name,
                            'team_type': team_type,
                            'price': outcome['price']
                        }
                        data_list.append(row)
    return pd.DataFrame(data_list)

def pivot_and_clean_data(df):
    """
    Pivots the DataFrame and fills NaN values with a placeholder.

    Args:
        df (pd.DataFrame): Raw DataFrame containing odds information.

    Returns:
        pd.DataFrame: Pivoted and cleaned DataFrame.
    """
    pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type'],
                                columns='bookmaker',
                                values='price',
                                aggfunc='first')
    pivoted_df.reset_index(inplace=True)
    pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
    pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
    pivoted_df.fillna('N/A', inplace=True)
    return pivoted_df

# Example usage
API_KEY = '287f26ecf5b58ab73781cfaf78310c08'
sport_key = 'americanfootball_ncaaf'
data = get_sport_odds(API_KEY, sport_key)
if data:
    processed_data = process_odds_data(data)
    pivoted_data = pivot_and_clean_data(processed_data)

def ev(dk, pin, bmgm, stake, bankroll):
    """
    Calculate expected value and optimal bet.

    Args:
        dk (tuple): A tuple containing odds for two events.
        pin (tuple): A tuple containing odds for event A and B from one sportsbook.
        bmgm (tuple): A tuple containing odds for event A and B from another sportsbook.
        stake (float): The stake amount.
        bankroll (float): The available bankroll.

    Returns:
        None
    """

    fair_pin = fair_odds(pin[0], pin[1])
    fair_bmgm = fair_odds(bmgm[0], bmgm[1])
    fair = (avg(fair_pin[0], fair_bmgm[0]), avg(fair_pin[1], fair_bmgm[1]))

    a_profit = profit(stake, dk[0])
    b_profit = profit(stake, dk[1])
    a_ev = a_profit * fair[0] - stake * fair[1]
    b_ev = b_profit * fair[1] - stake * fair[0]

    dkfair = fair_odds(dk[0], dk[1])

    if a_ev > b_ev:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_a = width(pin[0], pin[1])
        # print(f'odds: {dk[0]}')
        # print(f'width: {width(pin[0], pin[1])}')
        # print(f'ev: {a_ev}')
        # print(f'optimal_bet: ${round(bet, 2)}')
        return 0, a_ev, dk[0], bet, width_a
    else:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_b = width(pin[0], pin[1])
        # print(f'odds: {dk[1]}')
        # print(f'width: {width(pin[0], pin[1])}')
        # print(f'ev: {b_ev}')
        # print(f'optimal_bet: ${round(bet, 2)}')
        return 1, b_ev, dk[1], bet, width_b

test = pivoted_data[['date', 'sport', 'home_team', 'away_team', 'team', 'FanDuel', 'BetOnline.ag', 'Bovada']]
test['matchup'] = test['home_team'] + ' vs ' + test['away_team']
test['team_x'] = np.where(test['home_team'] == test['team'], 'home', 'away')
test = test.sort_values(['matchup', 'team_x'], ascending=False)


test['check'] = np.where((test['FanDuel'] == 'N/A') | (test['Bovada'] == 'N/A') | (test['BetOnline.ag'] == 'N/A'),1,0)
test = test[test['check'] == 0]
    
cols = ['FanDuel', 'BetOnline.ag', 'Bovada']
for col in cols:
    test[col] = test[col].astype('int')

consolidated_df = test.groupby(['date', 'sport', 'home_team', 'away_team']).agg({
    'FanDuel': tuple,
    'BetOnline.ag': tuple,
    'Bovada': tuple
}).reset_index()

result_df = consolidated_df.apply(lambda row: ev(row['FanDuel'], row['BetOnline.ag'], row['Bovada'], stake=100, bankroll=1500), axis=1, result_type='expand')

# Assign the resulting DataFrame columns to the final DataFrame
result_df.columns = ['team_index', 'ev', 'odds', 'bet', 'width']

# Concatenate the consolidated_df with the result_df
final_df = pd.concat([consolidated_df, result_df], axis=1)

final_df['team'] = np.where(final_df['team_index'] == 0, final_df['home_team'], final_df['away_team'])
final_df = final_df.sort_values('ev', ascending=False)

In [77]:
final_df

,date,sport,home_team,away_team,FanDuel,BetOnline.ag,Bovada,team_index,ev,odds,bet,width,team
16,2023-09-30T19:30:00Z,NCAAF,Nebraska Cornhuskers,Michigan Wolverines,"(800, -1400)","(640, -909)","(625, -1000)",0.0,17.977595,800.0,-1180.851064,31.0,Nebraska Cornhuskers
25,2023-09-30T22:30:00Z,NCAAF,Stanford Cardinal,Oregon Ducks,"(1600, -4500)","(1500, -5000)","(1200, -5000)",0.0,12.780299,1600.0,-1329.839704,0.0,Stanford Cardinal
32,2023-09-30T23:30:00Z,NCAAF,SMU Mustangs,Charlotte 49ers,"(-2100, 1060)","(-2000, 1000)","(-1600, 800)",1.0,11.781655,1060.0,1251.506024,0.0,Charlotte 49ers
19,2023-09-30T19:30:00Z,NCAAF,Toledo Rockets,Northern Illinois Huskies,"(-610, 440)","(-455, 360)","(-525, 375)",1.0,10.698096,440.0,968.031968,5.0,Northern Illinois Huskies
33,2023-09-30T23:30:00Z,NCAAF,Tennessee Volunteers,South Carolina Gamecocks,"(-490, 365)","(-417, 335)","(-420, 315)",1.0,4.992688,365.0,882.952763,18.0,South Carolina Gamecocks
13,2023-09-30T19:30:00Z,NCAAF,Marshall Thundering Herd,Old Dominion Monarchs,"(-720, 500)","(-588, 455)","(-650, 450)",1.0,4.257772,500.0,1021.400778,33.0,Old Dominion Monarchs
36,2023-10-01T00:00:00Z,NCAAF,UL Monroe Warhawks,Appalachian State Mountaineers,"(450, -630)","(420, -526)","(400, -575)",0.0,3.505560,450.0,-977.949940,6.0,UL Monroe Warhawks
12,2023-09-30T19:30:00Z,NCAAF,Georgia Tech Yellow Jackets,Bowling Green Falcons,"(-1800, 920)","(-1667, 900)","(-1600, 800)",1.0,2.729274,920.0,1218.657453,67.0,Bowling Green Falcons
6,2023-09-30T16:00:00Z,NCAAF,UConn Huskies,Utah State Aggies,"(164, -200)","(151, -172)","(150, -175)",0.0,1.967661,164.0,-413.043478,21.0,UConn Huskies
9,2023-09-30T18:30:00Z,NCAAF,Kent State Golden Flashes,Miami (OH) RedHawks,"(500, -720)","(485, -625)","(450, -650)",0.0,1.666488,500.0,-1021.400778,60.0,Kent State Golden Flashes


In [83]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime

def odds_conv(odds):
    """
    Convert odds to implied probabilities.
    """
    if odds > 0:
        return 1 / (odds / 100 + 1)
    else:
        return 1 / (1 + 100 / abs(odds))

def fair_odds(odds_a, odds_b):
    """
    Calculate fair odds for two events.
    """
    fair_a = odds_conv(odds_a) / (odds_conv(odds_a) + odds_conv(odds_b))
    fair_b = odds_conv(odds_b) / (odds_conv(odds_a) + odds_conv(odds_b))
    return (fair_a, fair_b)

def avg(x, y):
    """
    Calculate the average of two numbers.
    """
    return (x + y) / 2

def profit(stake, odds):
    """
    Calculate profit from a bet.
    """
    if odds > 0:
        return odds / 100 * stake
    else:
        return 100 / abs(odds) * stake

def kelly_criterion(b, p, q):
    """
    Calculate the Kelly Criterion for optimal betting.
    """
    return (b * p - q) / b

def width(a, b):
    """
    Calculate the width between two numbers.
    """
    if a < 0 and b < 0:
        return abs(int(str(a)[-2:]) + int(str(b)[-2:]))
    else:
        return abs(int(str(a)[-2:]) - int(str(b)[-2:]))

def get_sport_odds(api_key, sport_key='upcoming', regions='us', oddsFormat='american'):
    """
    Fetches sports odds data from the Odds API.
    """
    base_url = f'https://api.the-odds-api.com/v4/sports/{sport_key}/odds'
    params = {
        'apiKey': api_key,
        'regions': regions,
        'oddsFormat': oddsFormat,
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f'Failed to retrieve data. Status code: {response.status_code}')
        return None

def process_odds_data(data):
    """
    Processes the odds data and creates a DataFrame.
    """
    data_list = []
    for game in data:
        game_date = game['commence_time']
        sport = game['sport_title']
        home_team = game['home_team']
        away_team = game['away_team']
        for bookmaker in game['bookmakers']:
            bookmaker_name = bookmaker['title']
            for market in bookmaker['markets']:
                if market['key'] == 'h2h':
                    for outcome in market['outcomes']:
                        team_name = outcome['name']
                        if team_name.lower() == home_team.lower():
                            team_type = 'home'
                        elif team_name.lower() == away_team.lower():
                            team_type = 'away'
                        else:
                            team_type = 'unknown'
                        row = {
                            'date': game_date,
                            'sport': sport,
                            'home_team': home_team,
                            'away_team': away_team,
                            'bookmaker': bookmaker_name,
                            'team_type': team_type,
                            'price': outcome['price']
                        }
                        data_list.append(row)
    return pd.DataFrame(data_list)

def pivot_and_clean_data(df):
    """
    Pivots the DataFrame and fills NaN values with a placeholder.
    """
    pivoted_df = df.pivot_table(index=['date', 'sport', 'home_team', 'away_team', 'team_type'],
                                columns='bookmaker',
                                values='price',
                                aggfunc='first')
    pivoted_df.reset_index(inplace=True)
    pivoted_df['team_type'] = pivoted_df.apply(lambda row: row['away_team'] if row['team_type'] == 'away' else row['home_team'], axis=1)
    pivoted_df.rename(columns={'team_type': 'team'}, inplace=True)
    pivoted_df.fillna('N/A', inplace=True)
    return pivoted_df

def calculate_ev(dk, pin, bmgm, stake, bankroll):
    """
    Calculate expected value and optimal bet.
    """
    fair_pin = fair_odds(pin[0], pin[1])
    fair_bmgm = fair_odds(bmgm[0], bmgm[1])
    fair = (avg(fair_pin[0], fair_bmgm[0]), avg(fair_pin[1], fair_bmgm[1]))

    a_profit = profit(stake, dk[0])
    b_profit = profit(stake, dk[1])
    a_ev = a_profit * fair[0] - stake * fair[1]
    b_ev = b_profit * fair[1] - stake * fair[0]

    dkfair = fair_odds(dk[0], dk[1])

    if a_ev > b_ev:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_a = width(pin[0], pin[1])
        return 0, a_ev, dk[0], bet, width_a
    else:
        bet = bankroll * kelly_criterion(1, dkfair[0], dkfair[1])
        width_b = width(pin[0], pin[1])
        return 1, b_ev, dk[1], bet, width_b

def main(api_key, sport_key, regions, odds_format, stake, bankroll):
    """
    Main function to fetch, process, and analyze sports odds data.
    """
    data = get_sport_odds(api_key, sport_key, regions, odds_format)
    if data:
        processed_data = process_odds_data(data)
        pivoted_data = pivot_and_clean_data(processed_data)

        test = pivoted_data[['date', 'sport', 'home_team', 'away_team', 'team', 'FanDuel', 'BetOnline.ag', 'Bovada']]
        test['matchup'] = test['home_team'] + ' vs ' + test['away_team']
        test['team_x'] = np.where(test['home_team'] == test['team'], 'home', 'away')
        test = test.sort_values(['matchup', 'team_x'], ascending=False)

        test['check'] = np.where((test['FanDuel'] == 'N/A') | (test['Bovada'] == 'N/A') | (test['BetOnline.ag'] == 'N/A'), 1, 0)
        test = test[test['check'] == 0]
        
        cols = ['FanDuel', 'BetOnline.ag', 'Bovada']
        for col in cols:
            test[col] = test[col].astype('int')

        consolidated_df = test.groupby(['date', 'sport', 'home_team', 'away_team']).agg({
            'FanDuel': tuple,
            'BetOnline.ag': tuple,
            'Bovada': tuple
        }).reset_index()

        result_df = consolidated_df.apply(lambda row: calculate_ev(row['FanDuel'], row['BetOnline.ag'], row['Bovada'], stake, bankroll), axis=1, result_type='expand')

        result_df.columns = ['team_index', 'ev', 'odds', 'bet', 'width']
        final_df = pd.concat([consolidated_df, result_df], axis=1)

        final_df['team'] = np.where(final_df['team_index'] == 0, final_df['home_team'], final_df['away_team'])
        final_df = final_df.sort_values('ev', ascending=False)

        return final_df

# Example usage in the main() function
if __name__ == "__main__":
    API_KEY = '287f26ecf5b58ab73781cfaf78310c08'
    SPORT_KEY = 'americanfootball_ncaaf'
    REGIONS = 'us'
    ODDS_FORMAT = 'american'
    STAKE = 100
    BANKROLL = 1500
    final_dataframe = main(API_KEY, SPORT_KEY, REGIONS, ODDS_FORMAT, STAKE, BANKROLL)
    print(final_dataframe)


                    date  sport                           home_team  \
16  2023-09-30T19:30:00Z  NCAAF                Nebraska Cornhuskers   
25  2023-09-30T22:30:00Z  NCAAF                   Stanford Cardinal   
32  2023-09-30T23:30:00Z  NCAAF                        SMU Mustangs   
19  2023-09-30T19:30:00Z  NCAAF                      Toledo Rockets   
33  2023-09-30T23:30:00Z  NCAAF                Tennessee Volunteers   
13  2023-09-30T19:30:00Z  NCAAF            Marshall Thundering Herd   
36  2023-10-01T00:00:00Z  NCAAF                  UL Monroe Warhawks   
12  2023-09-30T19:30:00Z  NCAAF         Georgia Tech Yellow Jackets   
6   2023-09-30T16:00:00Z  NCAAF                       UConn Huskies   
9   2023-09-30T18:30:00Z  NCAAF           Kent State Golden Flashes   
1   2023-09-30T16:00:00Z  NCAAF                 Arkansas Razorbacks   
0   2023-09-30T16:00:00Z  NCAAF                          Akron Zips   
10  2023-09-30T19:00:00Z  NCAAF             California Golden Bears   
20  20